<a href="https://colab.research.google.com/github/zyang63/Flow-simulation/blob/main/flow_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
%%capture
try:
  import bpy
except ImportError:
  !pip install bpy
  import bpy
try:
  from stl import mesh
except ImportError:
  !pip install numpy-stl
  from stl import mesh
import numpy as np
from scipy import ndimage
from skimage import measure
import os
!apt-get install blender

In [117]:
blender_script = """
import bpy
import bmesh
from mathutils import Vector

bpy.ops.wm.read_factory_settings(use_empty=True)

# Import STL mesh and apply remesh modifier
bpy.ops.import_mesh.stl(filepath= "/content/Large_gate_assembly - Large_gate-1.STL", global_scale=0.1,)
bpy.ops.import_mesh.stl(filepath= "/content/Large_gate_assembly - casting_8x8x0.5-1.STL", global_scale=0.1)

# Get the active collection
collection = bpy.context.collection

# Create a new mesh to hold the merged geometry
merged_mesh = bpy.data.meshes.new(name="MergedMesh")
merged_object = bpy.data.objects.new("MergedObject", merged_mesh)

# Link the new object to the active collection
collection.objects.link(merged_object)

# Select all objects in the collection
bpy.ops.object.select_all(action='DESELECT')
for obj in collection.objects:
    obj.select_set(True)

# Set the active object to the last selected object (for merging)
bpy.context.view_layer.objects.active = collection.objects[-1]

# Join the selected objects into the active object
bpy.ops.object.join()

# Update the mesh to reflect the changes
merged_mesh.update()

# Deselect all objects
bpy.ops.object.select_all(action='DESELECT')

# Select the merged object
merged_object.select_set(True)

# Set the active object to the merged object
bpy.context.view_layer.objects.active = merged_object

obj = bpy.context.active_object

def create_hexahedron_from_geometry(obj):
    # Get the mesh data
    mesh = obj.data
    bm = bmesh.new()
    bm.from_mesh(mesh)

    # Initialize min and max coordinates
    min_coords = Vector((float('inf'), float('inf'), float('inf')))
    max_coords = Vector((-float('inf'), -float('inf'), -float('inf')))

    # Find min and max coordinates
    for vert in bm.verts:
        for i in range(3):
            min_coords[i] = min(min_coords[i], vert.co[i])
            max_coords[i] = max(max_coords[i], vert.co[i])

    # Calculate the size and location of the hexahedron
    size = max_coords - min_coords
    location = (min_coords + max_coords) / 2

    # Create a new cube
    bpy.ops.mesh.primitive_cube_add(size=1, location=(0, 0, 0))
    cube = bpy.context.object

    # Scale the cube to match the size of the hexahedron
    cube.scale = size *2

    # Move the cube to the center of the hexahedron
    cube.location = location

    # Apply a Boolean modifier to subtract the original geometry
    modifier = cube.modifiers.new(name="Boolean", type='BOOLEAN')
    modifier.operation = 'DIFFERENCE'
    modifier.use_self = True

    # Set the target object for the Boolean operation
    modifier.use_self = False
    modifier.use_self = True
    modifier.object = bpy.data.objects.get("MergedObject")

    # Apply the modifier to perform the boolean operation
    bpy.ops.object.modifier_apply({"object": cube}, modifier="Boolean")

    # Cleanup
    bm.free()

    bpy.ops.mesh.primitive_cube_add(size=1, location=(0, 0, 0))
    cube = bpy.context.object

    # Scale the cube to match the size of the hexahedron
    cube.scale = size *2.05

    # Move the cube to the center of the hexahedron
    cube.location = location

# Assume the active object is the input geometry
input_obj = bpy.data.objects.get("MergedObject")

# Check if the active object is a mesh
if input_obj and input_obj.type == 'MESH':
    # Create hexahedron based on the input geometry
    create_hexahedron_from_geometry(input_obj)
else:
    print("Select a mesh object as input.")

bpy.ops.import_mesh.stl(filepath="/content/pouring.stl", global_scale=10)

# Assuming you have a mesh named "pouring" and two cubes named "Cube" and "Cube.001"
pouring_obj = bpy.data.objects.get("pouring")
cube1_obj = bpy.data.objects.get("Cube")
cube2_obj = bpy.data.objects.get("Cube.001")

# Check if objects exist
if pouring_obj and cube1_obj and cube2_obj:
    # Set the active object to the pouring object
    bpy.context.view_layer.objects.active = pouring_obj

    # Boolean operation: Intersect with "Cube.001"
    bpy.ops.object.modifier_add(type='BOOLEAN')
    pouring_obj.modifiers["Boolean"].operation = 'INTERSECT'
    pouring_obj.modifiers["Boolean"].solver = 'FAST'
    pouring_obj.modifiers["Boolean"].use_self = True
    pouring_obj.modifiers["Boolean"].operand_type = 'OBJECT'
    pouring_obj.modifiers["Boolean"].object = cube2_obj

    # Apply the boolean modifier
    bpy.ops.object.modifier_apply({"object": pouring_obj}, modifier="Boolean")

    # Boolean operation: Subtract from "Cube"
    bpy.context.view_layer.objects.active = cube1_obj
    bpy.ops.object.modifier_add(type='BOOLEAN')
    cube1_obj.modifiers["Boolean"].operation = 'DIFFERENCE'
    cube1_obj.modifiers["Boolean"].use_self = True
    cube1_obj.modifiers["Boolean"].object = pouring_obj

    # Apply the boolean modifier
    bpy.ops.object.modifier_apply({"object": cube1_obj}, modifier="Boolean")


else:
    print("Objects not found.")

# Assuming you have a reference to the pouring object
#pouring_obj = bpy.context.active_object.bpy.data.objects.get("pouring")

# Get the current location of the object's origin
origin_location = pouring_obj.location.copy()

# Find the center of the geometry
bpy.ops.object.origin_set(type='ORIGIN_CENTER_OF_MASS', center='BOUNDS')
geometry_center = pouring_obj.location.copy()

# Set the scale factors for the X, Y, and Z axes
scale_x = 0.8
scale_y = 0.8
scale_z = 0.8

# Apply the scale transformation to the object
pouring_obj.scale = (scale_x, scale_y, scale_z)

# Calculate the difference in location after scaling
location_difference = geometry_center - pouring_obj.location

# Move the object to align the scaled geometry center with the original geometry center
pouring_obj.location += location_difference

file_path = "/content/Die.blend"
# Save the file
bpy.ops.wm.save_as_mainfile(filepath=file_path)"""
script_filename = "/content/Die_script.py"
with open(script_filename, "w") as script_file:
    script_file.write(blender_script)

# Run Blender in headless mode with the script
!blender -b -P {script_filename}

Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: scene view "Filmic" not found, setting default "Standard".
Import finished in 0.0008 sec.
Import finished in 0.0001 sec.
ERROR (bke.modifier): ./source/blender/blenkernel/intern/modifier.c:457 BKE_modifier_set_error: Object: "Cube", Modifier: "Boolean", Compiled without GMP, using fast solver
ERROR (bke.modifier): ./source/blender/blenkernel/intern/modifier.c:457 BKE_modifier_set_error: Object: "Cube", Modifier: "Boolean", Compiled without GMP, using fast solver
Import finished in 0.0005 sec.
ERROR (bke.modifier): ./source/blender/blenkernel/intern/modifier.c:457 BKE_modifier_set_error: Object: "Cube", Modifier: "Boolean", Compiled without GMP, using fast solver
ERROR (bke.modifier): ./source/blender/blenkernel/intern/modifier.c:457 BKE_modifier_set_error: Object: "Cube", 

In [116]:
blender_script = """
import bpy

file_path = "/content/Die.blend"

# Open the Blender file
bpy.ops.wm.open_mainfile(filepath=file_path)

# Set effector cube properties
effector_cube = bpy.data.objects.get("Cube")
if effector_cube:
    bpy.ops.object.select_all(action='DESELECT')
    effector_cube.select_set(True)
    bpy.context.view_layer.objects.active = effector_cube

    # Set domain properties
    bpy.ops.object.modifier_add(type='FLUID')
    effector_modifier = effector_cube.modifiers["Fluid"]
    effector_modifier.fluid_type = 'EFFECTOR'
else:
    print("Error: 'Cube' not found in the scene.")

# Set liquid domain properties
domain_cube = bpy.data.objects.get("Cube.001")
if domain_cube:
    bpy.ops.object.select_all(action='DESELECT')
    domain_cube.select_set(True)
    bpy.context.view_layer.objects.active = domain_cube

    # Set domain properties
    bpy.ops.object.modifier_add(type='FLUID')
    domain_modifier = domain_cube.modifiers["Fluid"]
    domain_modifier.fluid_type = 'DOMAIN'

    # Set domain settings
    domain_settings = domain_modifier.domain_settings
    domain_settings.domain_type = 'LIQUID'
    domain_settings.cache_directory = "/content/"
    domain_settings.resolution_max = 150
    domain_settings.cache_type = 'MODULAR'
    bpy.context.object.modifiers["Fluid"].domain_settings.cache_resumable = True
    bpy.context.object.modifiers["Fluid"].domain_settings.use_mesh = True
    bpy.context.object.modifiers["Fluid"].domain_settings.mesh_particle_radius = 2
    bpy.context.object.modifiers["Fluid"].domain_settings.cache_frame_end = 50

else:
    print("Error: 'Cube.001' not found in the scene.")

# Set pouring (inflow) object properties
inflow_obj = bpy.data.objects.get("pouring")
if inflow_obj:
    bpy.ops.object.select_all(action='DESELECT')
    inflow_obj.select_set(True)
    bpy.context.view_layer.objects.active = inflow_obj

    # Set inflow properties
    bpy.ops.object.modifier_add(type='FLUID')
    inflow_modifier = inflow_obj.modifiers["Fluid"]
    bpy.context.object.modifiers["Fluid"].fluid_type = 'FLOW'
    inflow_modifier.flow_settings.flow_type = 'LIQUID'
    bpy.context.object.modifiers["Fluid"].flow_settings.flow_behavior = 'INFLOW'
    bpy.context.object.modifiers["Fluid"].flow_settings.use_initial_velocity = True
    bpy.context.object.modifiers["Fluid"].flow_settings.velocity_coord[0] = 0
    bpy.context.object.modifiers["Fluid"].flow_settings.velocity_coord[1] = 0
    bpy.context.object.modifiers["Fluid"].flow_settings.velocity_coord[2] = -10

else:
    print("Error: 'Pouring' not found in the scene.")

# Bake the fluid simulation data
#bpy.ops.fluid.bake_data({'object': domain_cube})

# Bake the fluid simulation mesh
#bpy.ops.fluid.bake_mesh({'object': domain_cube})
#Save the Blender file
bpy.ops.wm.save_as_mainfile(filepath="/content/fluid_simulation.blend")"""
script_filename = "/content/Flow_script.py"
with open(script_filename, "w") as script_file:
    script_file.write(blender_script)
# Run Blender in headless mode with the script
!blender -b -P {script_filename}

Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /content/Die.blend
Info: Saved "fluid_simulation.blend"
Info: Saved "fluid_simulation.blend"
Writing: /tmp/fluid_simulation.crash.txt
